## Loading dataset 

In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [2]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

 93% 177M/191M [00:01<00:00, 120MB/s]
100% 191M/191M [00:01<00:00, 104MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 9.8 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 840 kB 41.7 MB/s 
     |████████████████████████████████| 3.4 MB 57.1 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 38.0 MB/s 
     |████████████████████████████████| 120 kB 55.6 MB/s 
     |████████████████████████████████| 234 kB 62.6 MB/s 
     |████████████████████████████████| 1.1 MB 57.2 MB/s 
     |████████████████████████████████| 6.4 MB 58.5 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 1.2 MB 45.4 MB/s 
     |████████████████████████████████| 25.3 MB 42.5 MB/s 
     |████████████████████████████████| 352 kB 60.9 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 9.1 MB/s 
     |████████████████████████████████| 462 kB 62.0 MB/s 
     |██████████████

In [5]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [6]:
train_image_dir = '/content/Final Img copy 2/train'
train_annotation_dir = '/content/Final Img copy 2/train anotation'
validate_image_dir = '/content/Final Img copy 2/valid'
validate_anaotation_dir = '/content/Final Img copy 2/valid anotation '

In [7]:

spec = model_spec.get('efficientdet_lite2')

In [8]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt"})

In [9]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt"})

In [10]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/10
176/176 [==============================] - 124s 385ms/step - det_loss: 1.1731 - cls_loss: 0.9191 - box_loss: 0.0051 - reg_l2_loss: 0.0769 - loss: 1.2500 - learning_rate: 0.0071 - gradient_norm: 3.4134 - val_det_loss: 0.7156 - val_cls_loss: 0.6668 - val_box_loss: 9.7651e-04 - val_reg_l2_loss: 0.0770 - val_loss: 0.7926
Epoch 2/10
176/176 [==============================] - 64s 364ms/step - det_loss: 0.6212 - cls_loss: 0.5315 - box_loss: 0.0018 - reg_l2_loss: 0.0771 - loss: 0.6983 - learning_rate: 0.0058 - gradient_norm: 3.7897 - val_det_loss: 0.5562 - val_cls_loss: 0.5322 - val_box_loss: 4.8096e-04 - val_reg_l2_loss: 0.0772 - val_loss: 0.6334
Epoch 3/10
176/176 [==============================] - 64s 367ms/step - det_loss: 0.5040 - cls_loss: 0.4399 - box_loss: 0.0013 - reg_l2_loss: 0.0772 - loss: 0.5813 - learning_rate: 0.0051 - gradient_norm: 3.3157 - val_det_loss: 0.4932 - val_cls_loss: 0.4697 - val_box_loss: 4.6964e-04 - val_reg_l2_loss: 0.0773 - val_loss: 0.5705
Epoch 4/10
1

## Evaluating the model 

In [11]:
model.evaluate(validate_dir,batch_size=5)

49/49 [==============================] - 16s 230ms/step



{'AP': 0.5623024,
 'AP50': 0.6482969,
 'AP75': 0.6157329,
 'AP_/Casual Shirt': 0.3045469,
 'AP_/Desgin sherwani': 0.6994577,
 'AP_/Design Kurta': 0.30914843,
 'AP_/Formal Shirt': 0.63008463,
 'AP_/Plain kurta': 0.4880242,
 'AP_/Plain sherwani': 0.6473443,
 'AP_/tshirt': 0.8575106,
 'APl': 0.5632886,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.8616061,
 'ARm': -1.0,
 'ARmax1': 0.8346547,
 'ARmax10': 0.8616061,
 'ARmax100': 0.8616061,
 'ARs': -1.0}

## Saving Model 

In [12]:
model.export(export_dir='/content/')